In [1]:
import boto3
import os
import keys

os.environ['AWS_ACCESS_KEY_ID'] = keys.AWS_ACCESS_KEY_ID
os.environ['AWS_SECRET_ACCESS_KEY'] = keys.AWS_SECRET_ACCESS_KEY
os.environ['AWS_DEFAULT_REGION'] = keys.AWS_DEFAULT_REGION

In [2]:
import calendar
import datetime
import json
import logging
import time
import boto3
from botocore.exceptions import ClientError
import requests

import Lambda.lambda_basics as lambda_basics
from api_gateway.api_gateway_rest import create_rest_api, construct_api_url, delete_rest_api

logger = logging.getLogger(__name__)


In [16]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
print('-'*88)
print("Setting up Lambda Function and API Gateway Rest API.")
print('-'*88)

----------------------------------------------------------------------------------------
Setting up Lambda Function and API Gateway Rest API.
----------------------------------------------------------------------------------------


In [17]:
lambda_filename = 'Lambda/lambda_handler_rest.py'
lambda_handler_name = 'Lambda/lambda_handler_rest.lambda_handler'
lambda_role_name = 'canaal-lambda-role'
lambda_function_name = 'canaal-moralis-lambda'
api_name = 'canaal-moralis-api'

In [18]:
iam_resource = boto3.resource('iam')
lambda_client = boto3.client('lambda')
apig_client = boto3.client('apigateway')

In [19]:
print(f"Creating AWS Lambda function {lambda_function_name} from "
      f"{lambda_handler_name}...")
deployment_package = lambda_basics.create_lambda_deployment_package(lambda_filename)
iam_role = lambda_basics.create_iam_role_for_lambda(iam_resource, lambda_role_name)
lambda_function_arn = lambda_basics.exponential_retry(
    lambda_basics.deploy_lambda_function, 'InvalidParameterValueException',
    lambda_client, lambda_function_name, lambda_handler_name, iam_role,
    deployment_package)

Creating AWS Lambda function canaal-moralis-lambda from Lambda/lambda_handler_rest.lambda_handler...


INFO: Created role canaal-lambda-role.
INFO: Attached basic execution policy to role canaal-lambda-role.
INFO: Attached basic execution policy to role canaal-lambda-role.
ERROR: Couldn't create function canaal-moralis-lambda.
Traceback (most recent call last):
  File "/Users/kaihayden/Desktop/Canaal/canaal/canaal-aws/Lambda/lambda_basics.py", line 135, in deploy_lambda_function
    response = lambda_client.create_function(
  File "/Users/kaihayden/opt/anaconda3/lib/python3.9/site-packages/botocore/client.py", line 401, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/Users/kaihayden/opt/anaconda3/lib/python3.9/site-packages/botocore/client.py", line 731, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.InvalidParameterValueException: An error occurred (InvalidParameterValueException) when calling the CreateFunction operation: The role defined for the function cannot be assumed by Lambda.


Sleeping for 1 to give AWS time to connect resources.


ERROR: Couldn't create function canaal-moralis-lambda.
Traceback (most recent call last):
  File "/Users/kaihayden/Desktop/Canaal/canaal/canaal-aws/Lambda/lambda_basics.py", line 135, in deploy_lambda_function
    response = lambda_client.create_function(
  File "/Users/kaihayden/opt/anaconda3/lib/python3.9/site-packages/botocore/client.py", line 401, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/Users/kaihayden/opt/anaconda3/lib/python3.9/site-packages/botocore/client.py", line 731, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.InvalidParameterValueException: An error occurred (InvalidParameterValueException) when calling the CreateFunction operation: The role defined for the function cannot be assumed by Lambda.


Sleeping for 2 to give AWS time to connect resources.


ERROR: Couldn't create function canaal-moralis-lambda.
Traceback (most recent call last):
  File "/Users/kaihayden/Desktop/Canaal/canaal/canaal-aws/Lambda/lambda_basics.py", line 135, in deploy_lambda_function
    response = lambda_client.create_function(
  File "/Users/kaihayden/opt/anaconda3/lib/python3.9/site-packages/botocore/client.py", line 401, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/Users/kaihayden/opt/anaconda3/lib/python3.9/site-packages/botocore/client.py", line 731, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.InvalidParameterValueException: An error occurred (InvalidParameterValueException) when calling the CreateFunction operation: The role defined for the function cannot be assumed by Lambda.


Sleeping for 4 to give AWS time to connect resources.


INFO: Created function 'canaal-moralis-lambda' with ARN: 'arn:aws:lambda:us-east-2:333152911718:function:canaal-moralis-lambda'.
INFO: Ran deploy_lambda_function, got arn:aws:lambda:us-east-2:333152911718:function:canaal-moralis-lambda.


In [20]:
print(f"Creating Amazon API Gateway REST API {api_name}...")
account_id = boto3.client('sts').get_caller_identity()['Account']
api_base_path = 'canaal-api'
api_stage = 'canaal-moralis-stage'
api_id = create_rest_api(
    apig_client, api_name, api_base_path, api_stage, account_id,
    lambda_client, lambda_function_arn)
api_url = construct_api_url(
    api_id, apig_client.meta.region_name, api_stage, api_base_path)
print(f"REST API created, URL is :\n\t{api_url}")
print(f"Sleeping for a couple seconds to give AWS time to prepare...")
time.sleep(2)

Creating Amazon API Gateway REST API canaal-moralis-api...


INFO: Create REST API canaal-moralis-api with ID k2sp7rs3c0.
INFO: Found root resource of the REST API with ID t7p4ajm9ol.
INFO: Created base path canaal-api with ID mslrcj.
INFO: Created a method that accepts all HTTP verbs for the base resource.
INFO: Set function arn:aws:lambda:us-east-2:333152911718:function:canaal-moralis-lambda as integration destination for the base resource.
INFO: Deployed REST API k2sp7rs3c0.
INFO: Granted permission to let Amazon API Gateway invoke function arn:aws:lambda:us-east-2:333152911718:function:canaal-moralis-lambda from arn:aws:execute-api:us-east-2:333152911718:k2sp7rs3c0/*/*/canaal-api.
INFO: Constructed REST API base URL: https://k2sp7rs3c0.execute-api.us-east-2.amazonaws.com/canaal-moralis-stage/canaal-api.


REST API created, URL is :
	https://k2sp7rs3c0.execute-api.us-east-2.amazonaws.com/canaal-moralis-stage/canaal-api
Sleeping for a couple seconds to give AWS time to prepare...


In [15]:
print("Deleting the REST API, AWS Lambda function, and security role...")
time.sleep(5)  # Short sleep avoids TooManyRequestsException.
lambda_basics.delete_lambda_function(lambda_client, lambda_function_name)
for pol in iam_role.attached_policies.all():
    pol.detach_role(RoleName=iam_role.name)
iam_role.delete()
print(f"Deleted role {iam_role.name}.")
delete_rest_api(apig_client, api_id)

Deleted role canaal-lambda-role.


INFO: Deleted REST API g3nuar22g8.
